<a href="https://colab.research.google.com/github/fursew05/DOE/blob/main/%EC%8B%A4%ED%97%98%EA%B3%84%ED%9A%8D%EB%B2%95_2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit_posthocs

# 2장 Experiment with a Single Factor

## CRD(Completely Randomized Design)
- 두 집단의 평균을 비교하기 위해서 CRD를 가정해야함

- CRD란?
실험단위들의 각 처리 그룹 배치와 실험 순서를 랜덤으로 정하여 실험을 설계하는 방법을 말한다.

  $y_{ij} = \mu + \tau_i + \varepsilon_{ij}, \quad \sum_{i=1}^{t}\tau_i=0$

## 일원배치 분산분석
CRD에서 고려하는 요인이 한가지인 경우에 사용하는 분산분석 방법이다.
예를 들어 어떤 모터의 수명을 종속변수로 두었을때 배터리의 종류가 2가지인 경우에는 t-test를 사용하면 되지만, 종류가 3가지 즉 3개 이상의
집단의 평균을 비교하는 경우 분산분석을 진행하게 된다.

이때 실험하게 되는 횟수는 요인의 level 수 * 샘플의 수가 된다
배터리 종류 3개에 대해서 모터의 수명시간을 10번씩 측정한다면 30회 실험 진행

In [ ]:
import pandas as pd

df = pd.read_csv('/content/sample_data/mtcars.csv',index_col=0)
df.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [ ]:
# t-test : gear가 3과 4인 집단의 disp 평균을 비교하고 싶은 경우 ttest_ind 함수 활용
from scipy.stats import ttest_ind
gear_3 = df[df['gear']==3]['disp']
gear_4 = df[df['gear']==4]['disp']

result = ttest_ind(gear_3,gear_4)
print(f"t통계량 : {result[0]}")
print(f"p값 : {result[1]}")

t통계량 : 6.949417317275277
p값 : 2.767535093193568e-07


유의수준 5%에서 귀무가설을 기각할 수 있어 gear_3와 gear_4의
평균의 차이가 있다는 것을 알 수 있음

- 귀무가설 : 집단 간 평균의 차이가 없다
- 대립가설 : 집단 간 평균의 차이가 있다

- 일원분산분석을 하는 경우 다음의 사전검정이 필요함


1.   정규성 확인 : 변수의 왜도 값이 절댓값 2를 넘지 않는지 확인
2.   등분산 검정(bartlett test 또는 levene test) : H0는 변수 간 분산의 차이가 없다


In [ ]:
# One way ANOVA : gear가 3,4,5 세 집단의 disp 평균을 비교하고 싶은 경우
from scipy.stats import skew, bartlett

# 정규성 확인
gear_5 = df[df['gear']==5]['disp']

normality_result_gear_3 = skew(gear_3)
normality_result_gear_4 = skew(gear_4)
normality_result_gear_5 = skew(gear_5)

print(f"gear_3의 왜도 : {normality_result_gear_3}")
print(f"gear_4의 왜도 : {normality_result_gear_4}")
print(f"gear_5의 왜도 : {normality_result_gear_5}")

#등분산성 확인
bartlett_result = bartlett(gear_3,gear_4,gear_5)
print(f"등분산 검정의 p값 : {bartlett_result[1]}")

gear_3의 왜도 : -0.2668791892901188
gear_4의 왜도 : -0.2003117299768987
gear_5의 왜도 : 0.40812918060558423
등분산 검정의 p값 : 0.010299628043890017


In [ ]:
# 일원배치 분산분석 : F 통계량을 통하여 검정진행
# 귀무가설 : 기어 개수에 따른 배기량 평균의 차이가 없다
# 대립가설 : 기어 개수에 따른 배기량 평균의 차이가 존재한다.

# 방법 1 : f_oneway
from scipy.stats import f_oneway
result = f_oneway(gear_3,gear_4,gear_5)
if result[1] < 0.05:
  print("귀무가설 기각")
else:
  print("귀무가설을 기각할 수 없음")

귀무가설 기각


In [ ]:
# 방법 2: ols 활용하여 anova 테이블 만들기
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# ols 함수 안에는 '종속변수 이름 ~ 설명변수 이름', 데이터프레임 두가지를 넣으면 됨
# 설명변수가 범주형일때에는 C(변수명) 형식으로 넣어야 함
model = ols('disp ~ C(gear)', df).fit()
anova_result = anova_lm(model)

print(anova_result)
if anova_result['PR(>F)'].iloc[0] < 0.05:
  print("귀무가설 기각")
else:
  print("귀무가설을 기각할 수 없음")

            df         sum_sq        mean_sq          F    PR(>F)
C(gear)    2.0  280220.630021  140110.315010  20.734399  0.000003
Residual  29.0  195964.164667    6757.384989        NaN       NaN
귀무가설 기각


- p값이 정해놓은 유의수준보다 작은 경우 귀무가설을 기각할 수 있게 됨

  따라서 gear 별로 disp의 평균에 통계적으로 유의미한 차이가 있다는 것을 알 수 있음

  하지만 어느 집단에서 배기량이 차이를 보이고 있는지는 알 수가 없음
  
  그렇기 때문에 **사후검정**를 통하여 정확한 결과를 도출해야함

## Contrasts
위에서 진행한 방법은 일원배치 분산분석 중 global test로 한가지 요인에 대해서

여러 집단의 평균 차이가 있다는 것은 알 수 있지만
차이가 어디서 나타나는지는 알 수가 없음.

Contrasts는 평균 벡터와 선형결합을 하게되는 벡터를 말한다
두 집단의 자료의 양이 같은 경우에는 **contrast의 합이 0이 되도록** 실시하며,

불균형의 경우에는 그룹별 자료의 수를 가중치로 두어 **가중치와 contrast의 내적이
0이 되도록** 실시한다.

$L = \sum_{i=1}^{k} c_i \mu_i, \quad \text{단, } \sum_{i=1}^{k} c_i = 0$

예시) 위에 예시에서 global F test를 진행했을때 차이가 있다는 것을 확인했지만
어떤 집단에서 차이를 보이는지 알기위해서 기어가 3인 집단과 기어가 5인 집단의
차이가 존재하는지 확인하고 싶음

이때 벡터 C (1,0,-1)과 평균 벡터 (mu_3,mu_4,mu_5)간의 선형결합 값을 모수로 두어 가설 검정을 진행

- 귀무가설 : 선형결합이 0이다
- 대립가설 : 선형결합이 0이 아니다


In [ ]:
# mu3-mu5 = 0인지 확인하는 예제
from statsmodels.stats.contrast import Contrast
import numpy as np

model = ols('disp ~ C(gear)', data = df).fit()
contrast_vector = np.array([1,0,-1])
model.t_test(contrast_vector)

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0           450.1200     56.156      8.016      0.000     335.269     564.971

- p값이 매우 작은것으로 보아 기어3 집단과 기어5 집단 간 차이가 있다는 것을  확인할 수 있음

# contrast의 경우에는 파이썬에서 지원하지 않어서 R을 사용했습니다.

In [ ]:
# orthogonal contrast : 서로 내적했을때 0이 되는 contrast들로 진행하여 다른 contrast가 영향을 주지 않도록 설계
# 이러한 방식으로 설계하면 처리제곱합이 정확하게 나누어져 집단 간 명확한 차이를 확인할 수 있게 됨
# contrast의 경우에는 파이썬에서 지원하는 패키지가 없어서 R을 사용했습니다.

# 1. 데이터 준비
data(mtcars)

# 2. 설명변수를 범주형 데이터로 변환
mtcars$gear <- factor(mtcars$gear)


# 3. Contrast 정의
contrast_matrix <- matrix(c(-2, 1, 1,  # 대비 1: 3단 vs (4단, 5단)의 평균
                            0, -1, 1), # 대비 2: 4단 vs 5단
                           ncol = 2)


# 4. Factor에 Contrast 할당
contrasts(mtcars$gear) <- contrast_matrix

# 5. 모델 생성 및 최종 결과 확인
model_lm <- aov(disp ~ gear, data = mtcars)

# summary.aov() 함수로 분할된 ANOVA 테이블을 확인합니다.
print("--- 분할된 최종 ANOVA 테이블 ---")
summary(model_lm,split=list(gear=list("trt1"=1,"trt2"=2)))

[1] "--- gear 변수에 할당된 대비 확인 ---"
  [,1] [,2]
3   -2    0
4    1   -1
5    1    1
[1] "--- 분할된 최종 ANOVA 테이블 ---"


             Df Sum Sq Mean Sq F value   Pr(>F)    
gear          2 280221  140110  20.734 2.56e-06 ***
  gear: trt1  1 257934  257934  38.171 9.78e-07 ***
  gear: trt2  1  22286   22286   3.298   0.0797 .  
Residuals    29 195964    6757                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

- 결과 해석

1번째 검정 결과 유의수준 5%에서 귀무가설 기각
따라서, 기어4와 기어5의 배기량 평균과 기어3의 배기량 평균 차이가 존재함

2번째 검정 결과 유의수준 5%에서 귀무가설 기각하지 못함
따라서, 기어4와 기어5의 배기량 평균에는 차이가 없음

즉, 기어3이 기어4와 기어5와 다르게 배기량 차이가 있다는 결론을 내릴 수 있음

## 여러 사후검정 방법들

- 처리 방식 즉 집단이 3개 이상인 경우에는 모든 실험의 error rate를 고려해야함
- 유의수준이 5%여도 집단이 15개여서 2개씩 비교하는 실험을 할 때 총 105번 실험을 해야하고 각각 유의수준을 5%로 두면 전체실험으로 봤을때에는 525%가 나옴
- 이를 해결하는 방법으로 Bonferroni, Tukey, Sheffe, Dunnet의 방법이 있다

In [ ]:
#1. Bonferroni : 유의수준을 a로 두었을때 실험의 횟수만큼 나누어 유의수준을 두는 방법

# ex) 유의수준 0.05일때, gear에 대한 다중 비교는 3번일어나므로 0.05/3 으로 유의수준을 보정하여 검정 진행

In [ ]:
#2. Tukey : 집단 간 표본수가 동일한 경우에 사용할 수 있는 사후 검정방법
# 단, Tukey 검정은 집단 간 데이터 수가 동일해야 하고 데이터 수가 많을수록 신뢰도가 올라감
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np

# 랜덤시드 고정
np.random.seed(30)

df2 = pd.DataFrame({
    'grade':[1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3],
    'score': np.random.randint(60,100,size=30)
})

# groups에는 설명변수, endog에는 종속변수, alpha에는 유의수준을 넣어주면 됨
result = pairwise_tukeyhsd(groups = df2['grade'], endog = df2['score'], alpha = 0.05)
print("-----------------Tukey test-------------------")
print(result)

-----------------Tuckey test-------------------
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2     -1.3 0.9676 -14.4777 11.8777  False
     1      3      7.9 0.3131  -5.2777 21.0777  False
     2      3      9.2 0.2123  -3.9777 22.3777  False
-----------------------------------------------------


In [ ]:
#3.Sheffe : 집단 간 데이터의 수가 다를 때 사용하는 방법
from scikit_posthocs import posthoc_scheffe

result = posthoc_scheffe(df,val_col='disp',group_col='gear')
print(result)

          4         3         5
4  1.000000  0.000003  0.209756
3  0.000003  1.000000  0.023986
5  0.209756  0.023986  1.000000


- 결과 해석
보여지는 행렬의 원소는 두 집단의 disp 평균이 같은지 Sheffe의 검정을 진행 했을때 p_value임. 정해놓은 유의수준과 비교해서 p_value가 더 작으면 평균이 같다는 것을 기각할 수 있게 됨


1.   3단 기어와 4단 기어는 배기량 차이가 존재하고(0.0000003 < 0.05)
2.   3단 기어와 5단 기어는 배기량 차이가 존재하고(0.023986 < 0.05)
3.   4단 기어와 5단 기어는 배기량 차이가 없다.(0.209756 > 0.05)

In [ ]:
#4.Dunnet : 실험군과 대조군간의 사후 검정에 사용됨
from scikit_posthocs import posthoc_dunnett

result = posthoc_dunnett(df,val_col='disp',group_col='gear',control = 3)
print(result)

          4         3         5
4       NaN  0.000001       NaN
3  0.000001       1.0  0.013139
5       NaN  0.013139       NaN


- 결과 해석

1. gear가 3인 집단은 4와 5에 비해서 배기량 평균 차이가 확연하게 드러남
2. gear가 5인 집단보다 4인 집단과의 배기량 차이가 더 유의한 차이를 보임

## 크루스칼 왈리스 검정 (비모수적 방법)
평균 대신 중앙값을 이용하여 진행

일원배치 분산 분석을 진행할 때 정규성 검정을 만족하지 않을때는 크루스칼 왈리스 검정을 진행

측정값은 대소 비교가 가능한 변수여야하고 독립이라는 가정이 있음

- 귀무가설 : 모든 집단의 중앙값이 같다
- 대립가설 : 적어도 하나의 집단에서 다른 중앙값이 나타난다.

In [7]:
import pandas as pd
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns = iris.feature_names)
df['class'] = iris.target
df['class'] = df['class'].map({0:"Setosa",1:"Versicolor",2:"Virginica"})
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [17]:
# class 집단 별로 sepal length 정규성을 만족하는지 확인
from scipy.stats import shapiro, bartlett

# 정규성 확인
Setosa_petal_width = df.loc[df['class']=='Setosa',"petal width (cm)"]
Versicolor_petal_width = df.loc[df['class']=='Versicolor',"petal width (cm)"]
Virginica_petal_width = df.loc[df['class']=='Virginica',"petal width (cm)"]
print(f"setosa p-value : {shapiro(Setosa_petal_width)[1]}")
print(f"versicolor p-value : {shapiro(Versicolor_petal_width)[1]}")
print(f"virginica p-value : {shapiro(Virginica_petal_width)[1]}")

setosa p-value : 8.658572739428681e-07
versicolor p-value : 0.027277803876105258
virginica p-value : 0.0869541872909336


- 유의수준 5%에서 setosa, versicolor는 정규성을 만족하지 못한다는 것을 알 수 있음
- 따라서, 모수 검정방법인 ANOVA 대신 크루스칼-왈리스 검정을 진행

In [21]:
from scipy.stats import kruskal

result = kruskal(Setosa_petal_width, Versicolor_petal_width, Virginica_petal_width)
print(f"p_value : {result[1]:.4f}")

p_value : 0.0000


- 유의수준 5%에서 귀무가설을 기각하게 되어 집단별로 중앙값의 차이가 있음을 확인할 수 있다